In [53]:
import pandas as pd
import geopandas as gpd
from siuba import *
import shared_utils

from calitp_data_analysis import get_fs, geography_utils
from segment_speed_utils import helpers, time_series_utils, gtfs_schedule_wrangling
from segment_speed_utils.project_vars import SCHED_GCS, SEGMENT_GCS, GTFS_DATA_DICT, analysis_date

i'd start with 5, checking if the columns you want are already there in this file (saved out at the end of stop_arrivals_to_speed, and is a speeds by trip table for that day)

In [2]:
catalog = shared_utils.catalog_utils.get_catalog('gtfs_analytics_data')

In [3]:
catalog.speedmap_segments

{'dir': '${gcs_paths.SEGMENT_GCS}', 'stage1': '${speeds_tables.vp_dwell}', 'proxy_stop_times': 'stop_time_expansion/speedmap_stop_times', 'stage2': 'nearest/nearest_vp_speedmap_proxy', 'stage3': 'speedmap/stop_arrivals_proxy', 'stage3b': 'speedmap/stop_arrivals', 'stage4': 'speedmap/speeds', 'trip_stop_cols': ['trip_instance_key', 'stop_sequence', 'stop_sequence1'], 'shape_stop_cols': ['shape_array_key', 'shape_id'], 'stop_pair_cols': ['stop_pair', 'stop_pair_name', 'segment_id'], 'route_dir_cols': ['route_id', 'direction_id'], 'segments_file': 'segment_options/speedmap_segments', 'shape_stop_single_segment': 'rollup_singleday/speeds_shape_speedmap_segments', 'shape_stop_single_segment_detail': 'rollup_singleday/speeds_shape_speedmap_segments_detail', 'route_dir_single_segment': 'rollup_singleday/speeds_route_dir_speedmap_segments', 'route_dir_multi_segment': 'rollup_multiday/speeds_route_dir_speedmap_segments', 'min_trip_minutes': '${speed_vars.time_min_cutoff}', 'max_trip_minutes': 1

In [4]:
analysis_date

'2024-12-11'

In [5]:
# path = f'{catalog.speedmap_segments.dir}{catalog.speedmap_segments.stage4}_{analysis_date}.parquet'

In [6]:
path = f'{catalog.speedmap_segments.dir}{catalog.speedmap_segments.shape_stop_single_segment_detail}_{analysis_date}.parquet'

In [7]:
path

'gs://calitp-analytics-data/data-analyses/rt_segment_speeds/rollup_singleday/speeds_shape_speedmap_segments_detail_2024-12-11.parquet'

In [8]:
detail = gpd.read_parquet(path)

In [9]:
detail >> head(3)

,schedule_gtfs_dataset_key,shape_array_key,shape_id,route_id,direction_id,stop_pair,stop_pair_name,segment_id,time_of_day,p50_mph,...,p80_mph,name,caltrans_district,organization_source_record_id,organization_name,base64_url,geometry,n_trips_sch,trips_hr_sch,route_short_name
0,015d67d5b75b5cf2b710bbadadfb75f5,43f0d67e5131502a51f9330e04bacc55,20,228,0.0,40512__40514,Sir Francis Drake Blvd & Olema Rd__Sir Francis...,40512-40514-1,AM Peak,11.83,...,11.83,Bay Area 511 Marin Schedule,04 - Oakland,recNOb7pqBRlQVG5e,Marin County Transit District,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,"LINESTRING (-122.60413 38.00022, -122.60416 38...",3,1.0,228
1,015d67d5b75b5cf2b710bbadadfb75f5,43f0d67e5131502a51f9330e04bacc55,20,228,0.0,40514__40516,Sir Francis Drake Blvd & Alhambra Circle__Sir ...,40514-40516-1,AM Peak,15.52,...,19.79,Bay Area 511 Marin Schedule,04 - Oakland,recNOb7pqBRlQVG5e,Marin County Transit District,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,"LINESTRING (-122.60045 37.99908, -122.60037 37...",3,1.0,228
2,015d67d5b75b5cf2b710bbadadfb75f5,43f0d67e5131502a51f9330e04bacc55,20,228,0.0,40516__40518,Sir Francis Drake Blvd At Drake Manor Apts__Si...,40516-40518-1,AM Peak,15.69,...,20.79,Bay Area 511 Marin Schedule,04 - Oakland,recNOb7pqBRlQVG5e,Marin County Transit District,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,"LINESTRING (-122.59907 37.99704, -122.59894 37...",3,1.0,228


## need trip-level (pre-aggregation) gdf to properly calculate metrics

In [10]:
path = f'{catalog.speedmap_segments.dir}{catalog.speedmap_segments.stage4}_{analysis_date}.parquet'

In [11]:
st4 = pd.read_parquet(path)

In [12]:
st4.columns

Index(['trip_instance_key', 'stop_sequence', 'stop_sequence1',
       'shape_array_key', 'stop_meters', 'arrival_time', 'arrival_time_sec',
       'subseq_arrival_time_sec', 'subseq_stop_meters', 'meters_elapsed',
       'sec_elapsed', 'speed_mph', 'schedule_gtfs_dataset_key', 'route_id',
       'direction_id', 'route_short_name', 'scheduled_service_minutes',
       'sched_rt_category', 'time_of_day', 'shape_id', 'segment_id',
       'stop_pair_name', 'stop_pair'],
      dtype='object')

In [13]:
st4 >> head(3)

,trip_instance_key,stop_sequence,stop_sequence1,shape_array_key,stop_meters,arrival_time,arrival_time_sec,subseq_arrival_time_sec,subseq_stop_meters,meters_elapsed,...,route_id,direction_id,route_short_name,scheduled_service_minutes,sched_rt_category,time_of_day,shape_id,segment_id,stop_pair_name,stop_pair
0,0001b91f4f2c8ace7b3ebbefd5e9465b,1,1.0,b1e9813cd5549488c1770dd31bfcc739,506.474524,2024-12-11 19:02:41,68561,68610.0,777.366754,270.892229,...,9,0.0,9,20.0,schedule_and_vp,PM Peak,26481,80550-80253-1,3rd St at Morgan St__West Third St at Wilson S...,80550__80253
1,0001b91f4f2c8ace7b3ebbefd5e9465b,2,2.0,b1e9813cd5549488c1770dd31bfcc739,777.366754,2024-12-11 19:03:30,68610,68637.0,1053.968008,276.601254,...,9,0.0,9,20.0,schedule_and_vp,PM Peak,26481,80253-80548-1,West Third St at Wilson St-Downtown SMART__W 3...,80253__80548
2,0001b91f4f2c8ace7b3ebbefd5e9465b,3,3.0,b1e9813cd5549488c1770dd31bfcc739,1053.968008,2024-12-11 19:03:57,68637,68671.0,1383.761588,329.793579,...,9,0.0,9,20.0,schedule_and_vp,PM Peak,26481,80548-80342-1,W 3rd at Pierson St__North Dutton Ave at West ...,80548__80342


## corridor specification

In [14]:
orgs = detail.drop_duplicates(subset=['organization_source_record_id', 'organization_name'])[['organization_source_record_id', 'organization_name']]

In [15]:
orgs[orgs.organization_name.str.contains('San Fr')]

,organization_source_record_id,organization_name
365619,rechaapWbeffO33OX,City and County of San Francisco
403280,recmatCuQAUrNcs8j,San Francisco Bay Area Water Emergency Transit...


In [117]:
def corridor_from_segments(
    speed_segments_gdf: gpd.GeoDataFrame,
    organization_source_record_id: str,
    shape_id: str,
    start_seg_id: str,
    end_seg_id: str
) -> gpd.GeoDataFrame:
    '''
    
    '''
    
    shape_filtered = speed_segments_gdf.query("organization_source_record_id == @organization_source_record_id & shape_id == @shape_id")
    
    shape_filtered = shape_filtered.assign(start_point = shape_filtered.geometry.apply(lambda x: x.boundary.geoms[0]),
                      end_point = shape_filtered.geometry.apply(lambda x: x.boundary.geoms[1])
                     )

    filter_ids = [start_seg_id, end_seg_id]
    current_seg_id = start_seg_id
    assert start_seg_id in shape_filtered.segment_id.values and end_seg_id in shape_filtered.segment_id.values

    for _ in shape_filtered.segment_id:
        if current_seg_id == end_seg_id: break
        current_end = shape_filtered.loc[shape_filtered['segment_id'] == current_seg_id]['end_point'].iloc[0]
        next_segment = shape_filtered.loc[shape_filtered['start_point'] == current_end]
        assert not next_segment.empty, f'unable to locate segment after {current_seg_id}'
        current_seg_id = next_segment.segment_id.iloc[0]
        filter_ids += next_segment.segment_id.to_list()
        
    relevant_segments = shape_filtered.query("segment_id in @filter_ids").drop_duplicates(subset='segment_id')
    corridor = relevant_segments.dissolve()[['schedule_gtfs_dataset_key', 'shape_array_key', 'shape_id',
                                            'name', 'organization_source_record_id', 'geometry']]
    corridor_start = corridor.geometry.iloc[0].boundary.geoms[0]
    corridor_end = corridor.geometry.iloc[0].boundary.geoms[1]
    print(corridor_start, corridor_end)
    corridor = corridor.to_crs(geography_utils.CA_NAD83Albers).assign(distance_meters = lambda x: x.geometry.length)
    corridor.geometry = corridor.buffer(100) #  100m corridor buffer
    
    return corridor

In [ ]:
## move to shared_utils/geo_utils...

In [ ]:
corridor_start = corridor.geometry.iloc[0].boundary.geoms[0]
corridor_end = corridor.geometry.iloc[0].boundary.geoms[1]

In [102]:
import pyproj
geodesic = pyproj.Geod(ellps="WGS84")

In [112]:
long1, lat1 = (-122.40550254785889, 37.76900326502991)
long2, lat2 = (-122.40217200000001, 37.724137999999975)

In [113]:
fwd_azimuth,back_azimuth,distance = geodesic.inv(long1, lat1, long2, lat2)

In [114]:
fwd_azimuth

176.62550029429215

In [115]:
back_azimuth

-3.3724608442238946

In [116]:
distance

4988.315323644854

In [109]:
# shape_id = '4953'
# start_seg_id = '18088-18089-1'
# end_seg_id = '16800-16806-1'

shape_id = '800'
start_seg_id = '14970-17900-2'
end_seg_id = '16357-16358-1'

In [118]:
gdf = corridor_from_segments(speed_segments_gdf=detail, organization_source_record_id=sf, shape_id=shape_id,
                      start_seg_id=start_seg_id, end_seg_id=end_seg_id)

POINT (-122.40550254785889 37.76900326502991) POINT (-122.40217200000001 37.724137999999975)


In [143]:
# gdf.explore()

In [80]:
gdf

,schedule_gtfs_dataset_key,shape_array_key,shape_id,name,organization_source_record_id,geometry,distance_meters
0,7cc0cb1871dfd558f11a2885c145d144,88039de0a52cdf6be3d52ebe6315a48f,800,Bay Area 511 Muni Schedule,rechaapWbeffO33OX,"POLYGON ((-211888.036 -28619.355, -211905.357 ...",5297.946134
